In [57]:
import datasets
import os
os.getcwd()

'/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset'

In [63]:
path = "./yunzhe/1002_submissions_results_H_Java.jsonl"
ds = datasets.load_dataset("json", data_files=path, split="train").filter(lambda x:x['lang']=='Java 8')
ds

Dataset({
    features: ['lang', 'source_code', 'tags', 'lang_cluster', 'src_uid', 'code_uid', 'difficulty', 'exec_outcome', 'task_id', 'hidden_unit_tests', 'unittests'],
    num_rows: 45
})

In [66]:
for d in ds:
    src_uid = d['src_uid']
    code_uid = d['code_uid']
    all_pass = True if all(item['exec_outcome'] == 'PASSED' for item in d['unittests']) else False
    print(f"{all_pass}")

True
False
True
False
False
False
True
True
True
True
True
True
False
False
False
True
True
True
True
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True


In [70]:
path = "./yunzhe/java_valid_code.jsonl"
ds_java = datasets.load_dataset("json", data_files=path, split="train").filter(lambda x:x['lang']=='Java 8')
print(ds_java)
# for d in ds_java:
#     src_uid = d['src_uid']
#     code_uid = d['code_uid']
#     # all_pass = d['pass_rate'] == 100
#     print(f"{src_uid}")
print(len(set(ds_java['src_uid'])))

Dataset({
    features: ['lang', 'source_code', 'lang_cluster', 'tags', 'code_uid', 'src_uid', 'difficulty', 'exec_outcome', 'task_id', 'unittests', 'is_correct'],
    num_rows: 106
})
106


In [ ]:
30/45

In [64]:
path = "./test_java.jsonl"
ds_mine = datasets.load_dataset("json", data_files=path, split="train").filter(lambda x:x['lang']=='Java 8')
ds_mine

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/45 [00:00<?, ? examples/s]

Dataset({
    features: ['lang', 'source_code', 'tags', 'lang_cluster', 'src_uid', 'code_uid', 'difficulty', 'exec_outcome', 'task_id', 'hidden_unit_tests', 'unittests', 'pass_rate', 'mean_cpu_time', 'mean_peak_mem'],
    num_rows: 45
})

In [67]:
for d in ds_mine:
    src_uid = d['src_uid']
    code_uid = d['code_uid']
    all_pass = d['pass_rate'] == 100
    print(f"{all_pass}")

True
True
False
False
False
False
True
True
True
True
True
True
False
False
True
True
True
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
True
True
True
False
True
False
False
True
False
True


In [47]:
import json
import datasets
import pandas as pd
import os
from pathlib import Path
import re
import logging
import datetime
# screen1
def screen_submi_by_focallang_with_testcases(focallang:list, submi_ds_path, testcase_ds, output_submi_ds_path):
    """从yunzhe的submi数据集中筛选出四种语言的提交，并选出其中有测试用例的，初步得到optimization任务的submi数据集"""
    srcuids_with_testcases = json.loads(open(testcase_ds).readline()).keys()
    with open(submi_ds_path) as in_f, open(output_submi_ds_path, "w") as out_f:
        for line in in_f:
            data = json.loads(line)
            if any(
                data['lang']==interested_lang for interested_lang in focallang
            ) and any(
                data['src_uid']==srcuid for srcuid in srcuids_with_testcases
            ):
                json.dump(data, out_f)
                out_f.write('\n')
def cal_subminum_per_srcuidlang(submi_ds_path, output_table_path):
    """将submi jsonl数据集转化为纵轴为srcuid，横轴为lang，entry为submi_num的表格并存储，方便人工检视数据"""
    submi_ds = datasets.load_dataset("json", data_files=str(submi_ds_path), split="train")
    submi_srcuids = set(submi_ds['src_uid'])
    submi_langs = set(submi_ds['lang'])
    subminum_per_srcuidlang_dict = {srcuid: {lang: 0 for lang in submi_langs} for srcuid in submi_srcuids}
    for data in submi_ds:
        subminum_per_srcuidlang_dict[data['src_uid']][data['lang']] += 1
    subminum_per_srcuidlang_df = pd.DataFrame(subminum_per_srcuidlang_dict).T
    subminum_per_srcuidlang_df.to_csv(output_table_path, index=True)
    return subminum_per_srcuidlang_dict

# screen2
def screen_ds_by_submicnt_and_testcasecnt_and_taskcnt(submi_per_srcuidlang_table_path, task_ds_path, submi_ds_path, testcase_ds_path, SUMBMI_CNT_THRESHOLD, TESTCASE_CNT_THRESHOLD, TASK_CNT_THRESHOLD, screened2_submi_ds_path, output_table_path):
    submi_per_srcuidlang_table = pd.read_csv(submi_per_srcuidlang_table_path, index_col=0)
    screened_srcuidlang_table = pd.DataFrame([],columns=['lang', 'srcuid', 'submi_cnt', 'testcase_cnt'])
    testcases_json = json.load(open(testcase_ds_path))
    # leverage submi_per_srcuidlang_table to build a table where (lang, srcuid) are the joint key, then use this table to execute screening
    for lang in submi_per_srcuidlang_table.columns:
        for srcuid in submi_per_srcuidlang_table.index:
            submi_cnt = submi_per_srcuidlang_table.loc[srcuid, lang]
            testcase_cnt = len(testcases_json[srcuid])
            if submi_cnt > SUMBMI_CNT_THRESHOLD and testcase_cnt > TESTCASE_CNT_THRESHOLD:
                screened_srcuidlang_table.loc[len(screened_srcuidlang_table)] = {'lang': lang, 'srcuid': srcuid, 'submi_cnt': submi_cnt, 'testcase_cnt': testcase_cnt}

    tasksnum_per_lang = screened_srcuidlang_table.groupby('lang')['srcuid'].count()
    for lang in screened_srcuidlang_table['lang'].unique():
        if tasksnum_per_lang.loc[lang] < TASK_CNT_THRESHOLD:
            screened_srcuidlang_table.drop(index=screened_srcuidlang_table[screened_srcuidlang_table['lang']==lang].index, inplace=True)
    screened_srcuidlang_table.to_csv(output_table_path, index=False)
    # then update datasets according to the screened table
    with open(task_ds_path) as in_f, open(save_dir / Path("screen2_tasks.jsonl"), "w") as out_f:
        for line in in_f:
            data = json.loads(line)
            if data['src_uid'] in screened_srcuidlang_table['srcuid'].unique():
                json.dump(data, out_f)
                out_f.write('\n')
    with open(testcase_ds_path) as in_f, open(save_dir / Path("screen2_testcases.jsonl"), "w") as out_f:
        data = json.load(in_f)
        output_data = {}
        for (srcuid, testcases) in data.items():
            if srcuid not in screened_srcuidlang_table['srcuid'].unique():
                output_data[srcuid] = testcases
        json.dump(data, out_f)
    with open(submi_ds_path) as in_f, open(screened2_submi_ds_path, "w") as out_f:
        for line in in_f:
            data = json.loads(line)
            if data['src_uid'] in screened_srcuidlang_table['srcuid'].unique() and data['lang'] in screened_srcuidlang_table['lang'].unique():
                json.dump(data, out_f)
                out_f.write('\n')
# sample
def sample_task_by_timememory(srcuidlang_table_path, task_ds_path, task_tmp_dir, sampled_task_ds_path, sample_threshold):
    if not os.path.exists(task_tmp_dir):
        os.mkdir(task_tmp_dir)

    parse_num = lambda s: float(re.findall(r"\d+", s)[0])
    srcuidlang_table = pd.read_csv(srcuidlang_table_path)
    for lang in srcuidlang_table['lang'].unique():
        task_df = pd.DataFrame([], columns=['src_uid', 'difficulty', 'time_limit(seconds)', 'memory_limit(megabytes)'])
        with open(task_ds_path) as f:
            for line in f:
                data = json.loads(line)
                if data['src_uid'] in srcuidlang_table[srcuidlang_table['lang']==lang]['srcuid'].unique():
                    task_df.loc[len(task_df)] = [data['src_uid'], parse_num(data['difficulty']), parse_num(data['time_limit']), parse_num(data['memory_limit'])]
        # 对task_df按difficulty，time_limit和memory_limit分布采样，每种语言采样50个task
        group_sample_num = 0
        while True:
            group_sample_num += 1
            sampled_df = task_df.groupby('time_limit(seconds)').apply(
                lambda x: x.groupby('memory_limit(megabytes)').apply(
                    lambda x: x.sample(n=min(group_sample_num,x.shape[0]))
                )
            )
            if sampled_df.shape[0] >=sample_threshold:
                break
        sampled_df.to_csv(str(task_tmp_dir / Path(f"{lang}_sampled_task.csv")), index=False)
    # 更新task数据集
    if os.path.exists(sampled_task_ds_path):
        os.remove(sampled_task_ds_path)
        print("deleted")
    files = os.listdir(task_tmp_dir)
    sampled_srcuids = set()
    for lang_srcuid_file in files: # 拿到每一个语言的筛选好的任务文件
        lang = lang_srcuid_file.split('_')[0]
        for sampled_srcuid in pd.read_csv(f"{task_tmp_dir}/{lang_srcuid_file}")['src_uid']:# 从筛选好的task文件中找到每一条srcuid
            sampled_srcuids.add(sampled_srcuid)

    with open(task_ds_path) as in_f, open(sampled_task_ds_path,'w') as out_f:
        for line in in_f:
            data = json.loads(line)
            if data['src_uid'] in sampled_srcuids:
                json.dump(data, out_f)
                out_f.write('\n')
def sample_submi_random(task_tmp_dir, screened_submi_ds_path, sampled_submi_ds_path, SAMPLE_NUM=None):
    if os.path.exists(sampled_submi_ds_path):
        os.remove(sampled_submi_ds_path)
        print("deleted")
    files = os.listdir(task_tmp_dir)

    for lang_srcuid_file in files: # 拿到每一个语言的筛选好的任务文件
        lang = lang_srcuid_file.split('_')[0]
        print(lang)
        for sampled_srcuid in pd.read_csv(f"{task_tmp_dir}/{lang_srcuid_file}")['src_uid']:# 从筛选好的task文件中找到每一条srcuid
            with open(screened_submi_ds_path) as in_f, open(sampled_submi_ds_path, 'a') as out_f: 
                sample_cnt = 0
                for line in in_f: # 对该题目的每一个题解，选取前SAMPLE_NUM个submi，作为最终数据集
                    data = json.loads(line)
                    if data['lang'] == lang and data['src_uid'] == sampled_srcuid:
                        json.dump(data, out_f)
                        out_f.write('\n')
                        sample_cnt+=1
                        if SAMPLE_NUM is not None:
                            if sample_cnt >= SAMPLE_NUM:
                                break

curdir = Path(os.path.abspath('')) # 如果是py文件，用Path(__file__).parent获取当前文件的目录路径
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
save_dir = curdir / Path(f"screened_sampled_dataset_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)# 如果多级目录不存在，递归创建

log_file_path = save_dir / Path("data_screening_sampling.log")
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter(fmt='%(asctime)s - %(filename)s - %(levelname)s - %(message)s',
                                datefmt='%Y-%m-%d %H:%M:%S')
file_handler = logging.FileHandler(filename=log_file_path, mode='w', encoding='utf-8')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.addHandler(stream_handler)


task_ds_path = "./yunzhe/descriptions_0822.jsonl"
submi_ds_path = "./yunzhe/valid_submission_samples.jsonl"
testcase_ds_path = "./yunzhe/testcases_0822.jsonl"

logging.info(f"start screening and sampling from:\n - task_ds:{task_ds_path}\n - submi_ds:{submi_ds_path}\n - testcase_ds:{testcase_ds_path}\nall the output files will be placed at:{save_dir}")

# screen 1: 从yunzhe的submi数据集中筛选出四种语言的提交，并选出其中有测试用例的，初步得到optimization任务的submi数据集
focal_langs = ['GNU C', 'GNU C++', 'Mono C#', 'Python 3']
screened1_submi_ds_path = save_dir / Path("screen1_submissions.jsonl")
screened1_submi_per_srcuidlang_table_path = save_dir / Path("screen1_submissions_per_srcuidlang.csv")
logging.info(f"screen1 start, screening submissions implemented by focal langs:{focal_langs}, and with testcases")
screen_submi_by_focallang_with_testcases(focal_langs, submi_ds_path, testcase_ds_path, screened1_submi_ds_path)
cal_subminum_per_srcuidlang(screened1_submi_ds_path, screened1_submi_per_srcuidlang_table_path)
logging.info(f"screen1 finished, output files:\n - screened1_submi_ds:{screened1_submi_ds_path}\n - screened1_submi_per_srcuidlang_table:{screened1_submi_per_srcuidlang_table_path}")

#screen 2: 从screen1的submi数据集中筛选出submi数大于10，testcase数大于20，task数大于10的，得到最终的submi数据集
SUMBMI_CNT_THRESHOLD = 4 # 一个task(srcuid)的submissions数大于该阈值才被考虑，因为submission多才较好的能体现人类水平分布
TESTCASE_CNT_THRESHOLD = 20 # 一个task(srcuid)的testcase数大于该阈值才被考虑，因为testcase多才能在我们平均的方法里表现算法的综合性能
TASK_CNT_THRESHOLD = 30 #100 一个lang下的task数大于该阈值才被考虑，因为task多才能表现LLM在该语言下的综合optimization性能
srcuidlang_table_path = save_dir / Path("screen2_srcuidlang_table.csv")
screened2_submi_ds_path = save_dir / Path("screen2_submissions.jsonl")
logging.info(f"screen2 start, screening submissions with submi_cnt_per_task>{SUMBMI_CNT_THRESHOLD}, testcase_cnt_per_task>{TESTCASE_CNT_THRESHOLD}, task_cnt_per_lang>{TASK_CNT_THRESHOLD}")
screen_ds_by_submicnt_and_testcasecnt_and_taskcnt(screened1_submi_per_srcuidlang_table_path, task_ds_path, screened1_submi_ds_path, testcase_ds_path, SUMBMI_CNT_THRESHOLD, TESTCASE_CNT_THRESHOLD, TASK_CNT_THRESHOLD, screened2_submi_ds_path, srcuidlang_table_path)
logging.info(f"screen2 finished, output files:\n - screened2_submi_ds:{screened2_submi_ds_path}\n - srcuidlang_table:{srcuidlang_table_path}")

#sample: 从screen2的task数据集中采样每种语言至少50个task，每个task采样10个submissions，得到最终的task和submi数据集
SAMPLE_TASK_NUM = 30 # 每种语言采样【至少】50个task(和TASK_CNT_THRESHOLD务必对应，至少不能比TASK_CNT_THRESHOLD大)
SAMPLE_SUBMI_NUM = None # 10 #每个task采样【准确】10个submissions
task_tmp_dir = save_dir / Path(f"sample{SAMPLE_TASK_NUM}task_per_lang")
screened_task_ds_path = save_dir / Path("screen2_tasks.jsonl")
sampled_task_ds_path = save_dir / Path("sampled_tasks.jsonl")
sampled_submi_ds_path = save_dir / Path("sampled_submissions.jsonl")
logging.info(f"sampling start, sampling {SAMPLE_TASK_NUM} tasks per lang, and {SAMPLE_SUBMI_NUM} submissions per task")
sample_task_by_timememory(srcuidlang_table_path, screened_task_ds_path, task_tmp_dir, sampled_task_ds_path, SAMPLE_TASK_NUM)
sample_submi_random(task_tmp_dir, screened2_submi_ds_path, sampled_submi_ds_path, SAMPLE_SUBMI_NUM)
logging.info(f"sampling finished, output files:\n - sampled_task_ds:{sampled_task_ds_path}\n - sampled_submi_ds:{sampled_submi_ds_path}")
logging.info(f"all the output files are placed at:{save_dir}")

2023-10-02 23:35:49 - 103545798.py - INFO - start screening and sampling from:
 - task_ds:./yunzhe/descriptions_0822.jsonl
 - submi_ds:./yunzhe/valid_submission_samples.jsonl
 - testcase_ds:./yunzhe/testcases_0822.jsonl
all the output files will be placed at:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49
2023-10-02 23:35:49 - 103545798.py - INFO - start screening and sampling from:
 - task_ds:./yunzhe/descriptions_0822.jsonl
 - submi_ds:./yunzhe/valid_submission_samples.jsonl
 - testcase_ds:./yunzhe/testcases_0822.jsonl
all the output files will be placed at:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49
2023-10-02 23:35:49 - 103545798.py - INFO - start screening and sampling from:
 - task_ds:./yunzhe/descriptions_0822.jsonl
 - submi_ds:./yunzhe/valid_submission_samples.jsonl
 - testcase_ds:./yunzhe/testcases_0822.jsonl
all the output files will be placed at:/

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

2023-10-02 23:36:08 - 103545798.py - INFO - screen1 finished, output files:
 - screened1_submi_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/screen1_submissions.jsonl
 - screened1_submi_per_srcuidlang_table:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/screen1_submissions_per_srcuidlang.csv
2023-10-02 23:36:08 - 103545798.py - INFO - screen1 finished, output files:
 - screened1_submi_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/screen1_submissions.jsonl
 - screened1_submi_per_srcuidlang_table:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/screen1_submissions_per_srcuidlang.csv
2023-10-02 23:36:08 - 103545798.py - INFO - screen1 finished, output files:
 - screened1_submi_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance

GNU C
Mono C#
Python 3
GNU C++


2023-10-02 23:38:13 - 103545798.py - INFO - sampling finished, output files:
 - sampled_task_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/sampled_tasks.jsonl
 - sampled_submi_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/sampled_submissions.jsonl
2023-10-02 23:38:13 - 103545798.py - INFO - sampling finished, output files:
 - sampled_task_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/sampled_tasks.jsonl
 - sampled_submi_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/sampled_submissions.jsonl
2023-10-02 23:38:13 - 103545798.py - INFO - sampling finished, output files:
 - sampled_task_ds:/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-10-02_23:35:49/sampled_tasks.jsonl
 - sa

In [61]:
len(set(focal_java_submi_ds['src_uid']))

130

In [88]:
def how_many_data_do_we_have(srcuidlang_table_path):
    srcuidlang_table = pd.read_csv(srcuidlang_table_path)
    inference_data_num = len(srcuidlang_table.index)# 有多少条task就有多少个inference数据
    baseline_exec_data_num = sum([submi_num * test_num for submi_num, test_num in zip(srcuidlang_table['submi_cnt'], srcuidlang_table['testcase_cnt'])])
    LLM_exec_data_num = sum([test_num for test_num in srcuidlang_table['testcase_cnt']])# 有多少个testcase就有多少个LLM推理结果需要执行的数据
    return inference_data_num, baseline_exec_data_num, LLM_exec_data_num

how_many_data_do_we_have(srcuidlang_table_path)

(160, 570456, 6523)

In [90]:
submi_ds = datasets.load_dataset("json", data_files=str(sampled_submi_ds_path), split="train")
submi_ds[0]

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 328.60it/s]
Generating train split: 320 examples [00:00, 6152.43 examples/s]


{'lang': 'GNU C',
 'source_code': '#include <stdio.h>\n\nint knights[100500], n;\n\nchar check(int k) {\n    int start, j;\n    if(n / k < 3)\n        return 0;\n    for(start = 0; start < k; start++) {\n        char isHappy = 1;\n        for(j = start; isHappy && j < n; j += k) {\n            if(knights[j] == 0) {\n                isHappy = 0;\n            }\n        }\n        if(isHappy)\n            return 1;\n    }\n    return 0;\n}\n\nint main() {\n    int i, j;\n    scanf("%d", &n);\n    for(i = 0; i < n; i++) {\n        scanf("%d", knights + i);\n    }\n    for(i = 1; (long long)(i) * i <= n; i++) {\n        if(n % i)\n            continue;\n        if(check(i) || check(n / i)) {\n            puts("YES");\n            return 0;\n        }\n    }\n    puts("NO");\n    return 0;\n}\n',
 'tags': ['dp', 'number theory', 'math'],
 'lang_cluster': 'C',
 'src_uid': 'd3a0402de1338a1a542a86ac5b484acc',
 'code_uid': '34184654fb16421248539e7699dd5232',
 'difficulty': 1600,
 'exec_outcome'

In [99]:
submi_ds[0]['unittests'][0]['output'][0]

'YES'

In [7]:
def get_testcases(code_uid):
    submi_ds = datasets.load_dataset("json", data_files=str(sampled_submi_ds_path), split="train")
    for submi in submi_ds:
        if submi['code_uid'] == code_uid:
            return submi['unittests']

In [8]:
def get_code(code_uid):
    submi_ds = datasets.load_dataset("json", data_files=str(sampled_submi_ds_path), split="train")
    for submi in submi_ds:
        if submi['code_uid'] == code_uid:
            return submi['source_code']

In [11]:
sampled_submi_ds_path = '/home/wyk/CodeLLMBenchmark/code_opt/code_performance_eval/dataset/screened_sampled_dataset_2023-09-20_17:52:54/sampled_submissions.jsonl'
get_testcases('1d3a8804e288dee710091aedda77299c')

[{'exec_outcome': 'PASSED',
  'input': '6 10\nalloc 5\nalloc 3\nerase 1\nalloc 6\ndefragment\nalloc 6\n',
  'output': ['1\n2\nNULL\n3\n'],
  'result': '1\n2\nNULL\n3'},
 {'exec_outcome': 'PASSED',
  'input': '6 1\ndefragment\nalloc 10\nalloc 1\nerase -1\nerase 1\nerase 1\n',
  'output': ['NULL\n1\nILLEGAL_ERASE_ARGUMENT\nILLEGAL_ERASE_ARGUMENT\n'],
  'result': 'NULL\n1\nILLEGAL_ERASE_ARGUMENT\nILLEGAL_ERASE_ARGUMENT'},
 {'exec_outcome': 'PASSED',
  'input': '14 100\nalloc 99\nalloc 1\nalloc 1\nerase 2\nalloc 1\nerase 4\nerase 1\nalloc 100\nalloc 1\nalloc 99\ndefragment\nerase 4\nalloc 100\nalloc 99\n',
  'output': ['1\n2\nNULL\n3\nILLEGAL_ERASE_ARGUMENT\nNULL\n4\nNULL\nNULL\nNULL\n'],
  'result': '1\n2\nNULL\n3\nILLEGAL_ERASE_ARGUMENT\nNULL\n4\nNULL\nNULL\nNULL'},
 {'exec_outcome': 'PASSED',
  'input': '26 25\ndefragment\nerase 1\nerase -1560200883\nalloc 44\ndefragment\nalloc 75\nalloc 22\ndefragment\nerase 4\ndefragment\nalloc 57\nalloc 53\nerase 4\nerase -1639632026\nerase -21216050

In [2]:
def get_task_description(src_uid):
    task_ds = datasets.load_dataset("json", data_files=str(sampled_task_ds_path), split="train")
    for task in task_ds:
        if task['src_uid'] == src_uid:
            return task['description']

In [120]:
get_code('1b7a2be826dca647e28f51af4015e53b')

'n = int(input())\na = list(map(int, input().split()))\nm = int(input())\nbf = list(map(int,input().split()))\nb = bf[::-1]\nrt = 0\nlt = {}\nfor i in a:\n    for j in b:\n        if j/i >= rt and j%i == 0:\n            rt = j/i\n            if rt in lt.keys():\n                lt[rt] += 1\n            else:\n                lt[rt] = 1\n#print(lt)\nres = max(lt ,key = int)\nprint(lt[res])'

In [121]:
sample_code = get_code('1b7a2be826dca647e28f51af4015e53b')
task_description = get_task_description('102667eaa3aee012fef70f4192464674')
prompt = f"""
As an expert software developer with years of experience, please meticulously inspect the following low performance code sample and give a optimized version of the code, making it implement the same exact function but achieve faster execution time or smaller memory usage. The generated optimized version of code should strictly follow the same dependencies information and input output logic as the sample source code.
The detailed information are as follows:
1. Description of the task which the sample code solves: {task_description}
2. sample source code: 
```
{sample_code}
```
Respond only with a string in the following JSON format:
{{"can_be_optimized": True or False, "optimization_ideas": ideas string, “optimized_code”: code string}}
"""
prompt

'\nAs an expert software developer with years of experience, please meticulously inspect the following low performance code sample and give a optimized version of the code, making it implement the same exact function but achieve faster execution time or smaller memory usage. The generated optimized version of code should strictly follow the same dependencies information and input output logic as the sample source code.\nThe detailed information are as follows:\n1. Description of the task which the sample code solves: Vasya\'s bicycle chain drive consists of two parts: n stars are attached to the pedal axle, m stars are attached to the rear wheel axle. The chain helps to rotate the rear wheel by transmitting the pedal rotation.We know that the i-th star on the pedal axle has ai (0<a1<a2<...<an) teeth, and the j-th star on the rear wheel axle has bj (0<b1<b2<...<bm) teeth. Any pair (i,j) (1≤i≤n;\xa01≤j≤m) is called a gear and sets the indexes of stars to which the chain is currently atta

In [122]:
print(prompt)


As an expert software developer with years of experience, please meticulously inspect the following low performance code sample and give a optimized version of the code, making it implement the same exact function but achieve faster execution time or smaller memory usage. The generated optimized version of code should strictly follow the same dependencies information and input output logic as the sample source code.
The detailed information are as follows:
1. Description of the task which the sample code solves: Vasya's bicycle chain drive consists of two parts: n stars are attached to the pedal axle, m stars are attached to the rear wheel axle. The chain helps to rotate the rear wheel by transmitting the pedal rotation.We know that the i-th star on the pedal axle has ai (0<a1<a2<...<an) teeth, and the j-th star on the rear wheel axle has bj (0<b1<b2<...<bm) teeth. Any pair (i,j) (1≤i≤n; 1≤j≤m) is called a gear and sets the indexes of stars to which the chain is currently attached. Ge